In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import torch
from torchvision.ops import nms
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

# === 1. Load mô hình YOLO đã fine-tune ===
model = YOLO('/kaggle/input/best-pt/best.pt')

# === 2. Đọc ảnh ===
img_path = '/kaggle/input/picture-1-2/Pic_1_2.jpg'
image_bgr = cv2.imread(img_path)
original = image_bgr.copy()

# === 3. TÁCH MÀU HẠT VÀ LOẠI NỀN ĐỎ ===
hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)

# Mask giữ màu hạt (vàng cam và hồng nhạt)
lower1 = np.array([5, 40, 80])
upper1 = np.array([25, 255, 255])
lower2 = np.array([0, 30, 70])
upper2 = np.array([10, 255, 255])
mask1 = cv2.inRange(hsv, lower1, upper1)
mask2 = cv2.inRange(hsv, lower2, upper2)
mask = cv2.bitwise_or(mask1, mask2)

# Mask nền đỏ nhạt chìm để loại bỏ
lower_redbg = np.array([0, 20, 50])
upper_redbg = np.array([5, 120, 150])
mask_redbg = cv2.inRange(hsv, lower_redbg, upper_redbg)

# Loại vùng đỏ ra khỏi mask
mask = cv2.bitwise_and(mask, cv2.bitwise_not(mask_redbg))

# Morphology
kernel = np.ones((5, 5), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
mask = cv2.dilate(mask, kernel, iterations=1)

# Áp mask để giữ lại vùng chứa hạt
image_bgr = cv2.bitwise_and(image_bgr, image_bgr, mask=mask)

# === 4. TIỀN XỬ LÝ CHÓI, GAMMA, SHARP ===
blurred = cv2.GaussianBlur(image_bgr, (3, 3), 0)
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv)
gray_mean = np.mean(v)

if gray_mean > 50:
    v[v > 180] = 250
    s = cv2.add(s, 30)
    v = cv2.subtract(v, 20)
else:
    v[v > 230] = 300

hsv = cv2.merge((h, s, v))
enhanced = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def adjust_gamma(img, gamma=1.3):
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(img, table)

enhanced = adjust_gamma(enhanced, gamma=1.3)
sharp = cv2.addWeighted(enhanced, 1.5, cv2.GaussianBlur(enhanced, (0, 0), 3), -0.5, 0)

# === 5. PAD ẢNH ===
pad = 160
sharp_padded = cv2.copyMakeBorder(sharp, pad, pad, pad, pad, borderType=cv2.BORDER_REFLECT)

# === 6. CHIA TILE VÀ DỰ ĐOÁN ===
H, W = sharp_padded.shape[:2]
tile_size = 896
stride = 512
all_boxes = []
all_scores = []

for y in range(0, H, stride):
    for x in range(0, W, stride):
        tile = sharp_padded[y:y+tile_size, x:x+tile_size]
        if tile.shape[0] < 100 or tile.shape[1] < 100:
            continue

        results = model(tile, conf=0.1, max_det=700, imgsz=tile_size)[0]
        boxes = results.boxes.xyxy.cpu().numpy()
        scores = results.boxes.conf.cpu().numpy()

        for box, score in zip(boxes, scores):
            x1, y1, x2, y2 = box
            w, h = x2 - x1, y2 - y1
            area = w * h
            ratio = max(w / h, h / w)
            if area < 100 or area > 4000 or ratio > 3.0:
                continue
            all_boxes.append([x1 + x - pad, y1 + y - pad, x2 + x - pad, y2 + y - pad])
            all_scores.append(score)

# === 7. NMS + LOẠI BOX GẦN MÉP ẢNH ===
image_height, image_width = original.shape[:2]
all_boxes = torch.tensor(all_boxes).float()
all_scores = torch.tensor(all_scores).float()
keep = nms(all_boxes, all_scores, iou_threshold=0.15)

filtered_boxes = []
for i in keep:
    x1, y1, x2, y2 = all_boxes[i].tolist()
    score = all_scores[i].item()
    if (x2 > image_width - 12 or y2 > image_height - 12) and score < 0.4:
        continue
    filtered_boxes.append([x1, y1, x2, y2])

final_boxes = torch.tensor(filtered_boxes)

# === 8. VẼ KẾT QUẢ ===
im_pil = Image.fromarray(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
draw = ImageDraw.Draw(im_pil)
for box in final_boxes:
    x1, y1, x2, y2 = box.tolist()
    draw.rectangle([x1, y1, x2, y2], outline='red', width=2)

# === 9. LƯU VÀ HIỂN THỊ ẢNH ===
output_path = "/kaggle/working/detected_result_2.jpg"
im_pil.save(output_path)
print(f"📸 Ảnh đã lưu tại: {output_path}")
print("⬇ Bạn có thể tải ảnh từ phần 'Output' ở thanh bên phải của Kaggle")

plt.figure(figsize=(10, 10))
plt.imshow(im_pil)
plt.title(f"✅ Số lượng hạt: {len(final_boxes)}", fontsize=16)
plt.axis('off')
plt.show()
